In [1]:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv('/content/gender_classification_v7.csv')
df

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female
...,...,...,...,...,...,...,...,...
4996,1,13.6,5.1,0,0,0,0,Female
4997,1,11.9,5.4,0,0,0,0,Female
4998,1,12.9,5.7,0,0,0,0,Female
4999,1,13.2,6.2,0,0,0,0,Female


In [3]:
# отбор нечисловых колонок (Категориальные признаки)

df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print(non_numeric_cols)
print(df_non_numeric)

['gender']
      gender
0       Male
1     Female
2       Male
3       Male
4     Female
...      ...
4996  Female
4997  Female
4998  Female
4999  Female
5000    Male

[5001 rows x 1 columns]


In [4]:
# отбор числовых колонок (Числовые признаки)

df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)
print(df_numeric)

['long_hair' 'forehead_width_cm' 'forehead_height_cm' 'nose_wide'
 'nose_long' 'lips_thin' 'distance_nose_to_lip_long']
      long_hair  forehead_width_cm  forehead_height_cm  nose_wide  nose_long  \
0             1               11.8                 6.1          1          0   
1             0               14.0                 5.4          0          0   
2             0               11.8                 6.3          1          1   
3             0               14.4                 6.1          0          1   
4             1               13.5                 5.9          0          0   
...         ...                ...                 ...        ...        ...   
4996          1               13.6                 5.1          0          0   
4997          1               11.9                 5.4          0          0   
4998          1               12.9                 5.7          0          0   
4999          1               13.2                 6.2          0          0   


In [5]:
df["gender"].value_counts().sort_index()

Female    2501
Male      2500
Name: gender, dtype: int64

In [6]:
#OrdinalEncoder
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df)
categorical_columns

['gender']

In [7]:
df_cat = df[categorical_columns] #отберем только те столбцы в которых присутствует категориальная перемнная
df_cat.head()

,gender
0,Male
1,Female
2,Male
3,Male
4,Female


In [8]:
from sklearn.preprocessing import OrdinalEncoder

education_column = df_cat[["gender"]]

print(df_cat['gender'].unique()) #посмотрим какие виды образования присутствуют в выборке

encoder = OrdinalEncoder()   # вызываем кодировщик
encoder  = encoder.fit(df_cat[['gender']])
df_cat['gender'] = encoder.transform(df_cat[['gender']])
df_cat


['Male' 'Female']


<ipython-input-8-f44504ae0568>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['gender'] = encoder.transform(df_cat[['gender']])


,gender
0,1.0
1,0.0
2,1.0
3,1.0
4,0.0
...,...
4996,0.0
4997,0.0
4998,0.0
4999,0.0


In [9]:
print(encoder.categories_)
print(len(encoder.categories_[0]))
print(df_cat['gender'].unique())
df_cat['gender'].unique().size

[array(['Female', 'Male'], dtype=object)]
2
[1. 0.]


2

In [10]:
# OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoder  = encoder.fit(df_cat[['gender']])
CO_encoded  = encoder.transform(df_cat[['gender']])
CO_encoded

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [11]:
feature_names = encoder.get_feature_names_out(input_features=["gender"]) # получаем виды образования и на их основе создаем столбцы
CO_encoded = pd.DataFrame(CO_encoded, columns=feature_names)
CO_encoded

,gender_0.0,gender_1.0
0,0.0,1.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,1.0,0.0
...,...,...
4996,1.0,0.0
4997,1.0,0.0
4998,1.0,0.0
4999,1.0,0.0


In [12]:
df_cat_new=df_cat.join(CO_encoded)
df_cat_new=df_cat_new.drop(df_cat_new[['gender']],axis=1)
df_cat_new

,gender_0.0,gender_1.0
0,0.0,1.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,1.0,0.0
...,...,...
4996,1.0,0.0
4997,1.0,0.0
4998,1.0,0.0
4999,1.0,0.0


In [13]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()   # вызываем кодировщик
encoder  = encoder.fit(df_cat[['gender']])
df_cat['gender'] = encoder.transform(df_cat[['gender']])
df_cat

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
<ipython-input-13-f8be1d5e4c46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['gender'] = encoder.transform(df_cat[['gender']])


,gender
0,1
1,0
2,1
3,1
4,0
...,...
4996,0
4997,0
4998,0
4999,0


In [14]:
print(encoder.classes_)
print(len(encoder.classes_))
print(df_cat['gender'].unique())
df_cat['gender'].unique().size

[0. 1.]
2
[1 0]


2

In [32]:
X=df.iloc[:,1:3]
y=df.iloc[:,7]
df

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female
...,...,...,...,...,...,...,...,...
4996,1,13.6,5.1,0,0,0,0,Female
4997,1,11.9,5.4,0,0,0,0,Female
4998,1,12.9,5.7,0,0,0,0,Female
4999,1,13.2,6.2,0,0,0,0,Female


In [33]:
from sklearn.model_selection import train_test_split # подключаем функцию для разделения данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [34]:
# Масштабирование признаков
from sklearn.preprocessing import StandardScaler
import numpy as np
#X_train = np.array(df['forehead_width_cm'])

scaler = StandardScaler()
scaler.fit(X_train)
print('Математическое ожидание')
print(scaler.mean_)

print('Дисперсия')
print(scaler.var_)

print('Преобразованный набор')
print(scaler.transform(X_train))


Математическое ожидание
[13.18465714  5.94762857]
Дисперсия
[1.25299031 0.2937458 ]
Преобразованный набор
[[-0.6116448  -1.37943123]
 [ 0.46038631  0.2811368 ]
 [ 1.26440965  1.94170484]
 ...
 [-0.52230887  0.65015192]
 [ 0.72839409 -1.56393879]
 [-0.07562924  0.2811368 ]]


In [35]:
from sklearn.preprocessing import MinMaxScaler
#X_train = df['forehead_width_cm']

min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax

array([[0.26829268, 0.05      ],
       [0.56097561, 0.5       ],
       [0.7804878 , 0.95      ],
       ...,
       [0.29268293, 0.6       ],
       [0.63414634, 0.        ],
       [0.41463415, 0.5       ]])

In [36]:
# MaxAbsScaler

from sklearn.preprocessing import MaxAbsScaler


max_abs_scaler = MaxAbsScaler()
X_train_maxabs = max_abs_scaler.fit_transform(X_train)
X_train_maxabs

array([[0.80645161, 0.73239437],
       [0.88387097, 0.85915493],
       [0.94193548, 0.98591549],
       ...,
       [0.81290323, 0.88732394],
       [0.90322581, 0.71830986],
       [0.84516129, 0.85915493]])

In [39]:
# Нормализация
from sklearn import preprocessing


X_normalized = preprocessing.normalize(X_train, norm='l2')
X_normalized

array([[0.92329531, 0.38409085],
       [0.91353636, 0.40675707],
       [0.90171572, 0.43232946],
       ...,
       [0.89442719, 0.4472136 ],
       [0.93959732, 0.34228188],
       [0.90653641, 0.42212764]])